In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
briefly_url = 'https://briefly.ru'

In [3]:
def get_page(url):
    for i in range(5):
        r = requests.get(url)
        if r.status_code == 200:
            break
    
    if r.status_code != 200:
        print('error on {0}: status code = {1}'.format(url, r.status_code))
        
    return r.text

In [4]:
def get_authors_links():
    authors_url = 'https://briefly.ru/authors/'
    soup = BeautifulSoup(get_page(authors_url), 'lxml')
    
    authors = soup.find_all('div', class_='letter')[4] # на Д
    
    links = authors.find_all('a')
    links = map(lambda tag : tag['href'], links)
    links = filter(lambda link : not link.startswith('/surnames/'), links)
    links = map(lambda link : briefly_url + link, links)
    
    return list(links)

In [19]:
def get_summaries_links(author_url):
    soup = BeautifulSoup(get_page(author_url), 'lxml')
    
    author_name = soup.find('span', class_="author_name long")
    if author_name == None:
        author_name = soup.find('span', class_="author_name normal")
    
    author_name = author_name.text
    
    summaries_block = soup.find('section', class_='works_index')
    
    if summaries_block == None:
        return author_name, []
        
    links = summaries_block.find_all('a')
    links = map(lambda tag : tag['href'], links)
    links = filter(lambda link : link[0]=='/', links)
    links = map(lambda link : briefly_url + link, links)
    
    
    return author_name, list(links)

In [20]:
def get_summary(summary_url):
    soup = BeautifulSoup(get_page(summary_url), 'lxml')

    book_name = soup.find('span', class_="main").text
    
    full = soup.find('div', id='text')
    text_blocks = full.find_all('p')
    text_blocks = map(lambda tag : tag.text, text_blocks)
    
    text_blocks = map(lambda text : text.replace(u'\xa0', u' '), text_blocks)
    text_blocks = map(lambda text : text.replace(u'\u2009', u' '), text_blocks)

    return book_name, list(text_blocks)

In [21]:
def write_summary(file_id, summary):
    path = 'summaries/' + str(file_id) + '.txt'
    with open(path, 'w') as f:
        for block in summary:
            f.write(block)
            f.write('\n')

In [15]:
author_links = get_authors_links()

In [23]:
df = pd.DataFrame(columns=['author', 'book', 'file'])

In [33]:
file_id = 0

In [34]:
for author_url in author_links:
    author_name, summaries_links = get_summaries_links(author_url)
    if len(summaries_links) == 0:
        continue
    
    for link in summaries_links:
        book_name, summary = get_summary(link)
        write_summary(file_id, summary)
        df.loc[len(df)] = [author_name, book_name, file_id]
        file_id +=1

In [38]:
df

,author,book,file
0,Джеральд Даррелл,Моя семья и другие звери,0
1,Гавриил Романович Державин,Записки,1
2,Джеймс Джойс,Портрет художника в юности,2
3,Джеймс Джойс,Улисс,3
4,Чарльз Диккенс,Большие надежды,4
...,...,...,...
96,Александр Дюма,Виконт де Бражелон,96
97,Александр Дюма,Граф Монте-Кристо,97
98,Александр Дюма,Двадцать лет спустя,98
99,Александр Дюма,Королева Марго,99


In [39]:
df.to_csv('table.csv')